<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/hyper_Parameter_%EC%96%91%ED%8C%8C_%EC%8B%9C%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from xgboost import XGBRegressor


import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

양파

In [ ]:
dataset = pd.read_excel('/content/양파_시장.xlsx')
# dataset = pd.merge(dataset, wholesale, how='left', on='일자' )
# dataset.head()
# dataset = pd.read_excel('/content/무_특_반입_경락_시장_날씨_Holy_유가.xlsx')
# dataset = dataset.drop(['일자','품목명','품목코드','유무','평균 5cm 지중온도(°C)','평균 지면온도(°C)','최고기온(°C)','합계 일조시간(hr)','최저기온(°C)'], axis=1)
# dataset = dataset.drop(['일자', '최대 풍속(m/s)',	'평균 풍속(m/s)',	'평균 상대습도(%)',	'합계 일조시간(hr)',	'평균 지면온도(°C)',	'평균 5cm 지중온도(°C)', '여부'], axis=1)
# dataset = dataset[['도매가격', '100g가격', '유가 전국평균가격', '평균 풍속(m/s)', '여부', '최고기온(°C)', '평균 지면온도(°C)', '평균 5cm 지중온도(°C)', '일강수량(mm)', '반입량']]
# dataset = dataset.drop(['일자','Unnamed: 0'], axis=1)
#dataset = dataset.drop(['일자'], axis=1)
dataset = dataset.dropna()
dataset.head()
# dataset.corr()

In [ ]:
dataset.info()

In [ ]:
양파_더미 = pd.read_excel('/content/양파_더미데이터.xlsx')

In [ ]:
양파_더미.drop(['일자','가격'], axis=1, inplace=True)

In [ ]:
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import math

In [ ]:
dataset.corr()

In [ ]:
dataset2=dataset[['가격','경락가평균가격','반입량','유가 전국평균가격','도매가격']]
dataset2.head() #위 변수들만 dataset2로 지정 log 변환해줌

dataset2['가격']= np.log1p(dataset2['가격'])
dataset2['경락가평균가격']= np.log1p(dataset2['경락가평균가격'])
dataset2['반입량']= np.log1p(dataset2['반입량'])
dataset2['유가 전국평균가격']= np.log1p(dataset2['유가 전국평균가격'])
dataset2['도매가격']= np.log1p(dataset2['도매가격'])

dataset1=dataset[['유무','최저기온(°C)','최고기온(°C)','일강수량(mm)']]

from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
dataset1[:]=scalar.fit_transform(dataset1[:]) #전부다 MinMax로 

dataset=pd.concat([dataset1,dataset2],axis=1)
dataset.head()

In [ ]:
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)

In [ ]:
#sns.distplot(y_target, rug=True) 

In [ ]:
#dataset['가격']=np.log1p(dataset['가격'])
#sns.distplot(dataset['가격'],fit=norm)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
print(X_train.shape, X_test.shape)

In [ ]:
# 검증해주는 함수
def get_model_cv_prediction(model, X_data, y_target):
  neg_mse_scores=cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 3)
  rmse_scores = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)
  model.fit(X_data, y_target)
  score = model.score(X_data, y_target)
  kf_cv_scores = cross_val_score(model, X_data, y_target)
  print('#### ', model.__class__.__name__, ' ####')
  print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print('score : ', score)
  print("K-fold CV scores : {}" .format(kf_cv_scores))
  print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

## XGBoost Regressor 하이퍼파라미터


In [ ]:
# XGBoost Hypter Parameter Tuning
def XGBhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror'],
        'gamma' : [0.1, 0.2, 0.3, 0.4],

    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def RFRhyperParameterTuning(X_train, y_train):
    param_tuning = {
    'bootstrap': [True],
    'max_depth': [3, 5, 7, 10],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000],
    
    }

    rf_model = RandomForestRegressor()

    gsearch = GridSearchCV(estimator = rf_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def LGBMhyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.03, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_samples' : [10, 20, 30],
        'subsample' : [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['regression'],
        'gamma' : [0.1, 0.2, 0.3],
        'num_leaves' : [6, 8, 10]

    }

    lgbm = LGBMRegressor()

    gsearch = GridSearchCV(estimator = lgbm,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

def GBRhyperParameterTuning(X_train, y_train):
    
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.5, 0.7],
        'n_estimators' : [100, 200, 500]

    }

    gb_model = GradientBoostingRegressor()

    gsearch = GridSearchCV(estimator = gb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 3,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [ ]:
xgb_parameter = XGBhyperParameterTuning(X_train, y_train)
xgb_parameter

In [ ]:
rf_parameter = RFRhyperParameterTuning(X_train, y_train)
rf_parameter

In [ ]:
lgbm_parameter = LGBMhyperParameterTuning(X_train, y_train)
lgbm_parameter

In [ ]:

#rf_parameter = RFRhyperParameterTuning(X_train, y_train)
#rf_parameter

gbr_parameter= GBRhyperParameterTuning(X_train, y_train)
gbr_parameter


In [ ]:
# 모델 생성 및 예측
xgb_reg = XGBRegressor(colsample_bytree = 0.5, 
                       gamma = 0.1,
                       learning_rate = 0.01,
                       max_depth = 3,
                       min_child_weight = 5,
                       n_estimators = 500,
                       objective = 'reg:squarederror',
                       subsample = 0.5)

rf_reg = RandomForestRegressor(random_state=0, 
                               bootstrap = [True],
                               max_depth = 7,
                               max_features = 2,
                               min_samples_leaf = 5,
                               min_samples_split = 12,
                               n_estimators = 100
                               )

lgb_reg = LGBMRegressor(colsample_bytree = 0.5,
                        gamma = 0.1,
                        learning_rate = 0.01,
                        max_depth = 5,
                        min_child_samples = 10,
                        n_estimators = 200,
                        num_leaves =6,
                        objective = 'regression',
                        subsample = 0.5)

gb_reg = GradientBoostingRegressor(random_state=0, 
                                   learning_rate=0.01,
                                   max_depth=3,
                                   n_estimators=200,
                                   subsample=0.5
                                   )

models = [lgb_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]

for model in models:
  get_model_cv_prediction(model, X_train, y_train)

In [ ]:
ypred = rf_reg.predict(X_test)
mse = mean_squared_error(y_test, ypred)

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, ypred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

#rf_reg = RandomForestRegressor(random_state=0, 
#                              bootstrap = [True],
#                              max_depth = 7,
#                              max_features = 2,
#                               min_samples_leaf = 5,
#                               min_samples_split = 12,
#                               n_estimators = 100
#                               )

#gb_reg = GradientBoostingRegressor(random_state=0, 
#                                   learning_rate=0.01,
#                                   max_depth=3,
#                                   n_estimators=200,
#                                   subsample=0.5
#                                   )
er = VotingRegressor([('xgb_reg', xgb_reg), ('rf_reg', rf_reg)])
print(er.fit(X_train, y_train).predict(X_test))
y_pred = er.fit(X_train, y_train).predict(X_test)
er.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
#y_pred = er.fit(X_data, y_target).predict(X_data) 

In [ ]:
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="Original Price")
plt.plot(x_ax, y_pred, label="Predicted Price")
plt.title("Price Prediction")
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
y_target = dataset['가격']
X_data = dataset.drop(['가격'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.15, random_state=140)
estimators= ([('xgb_reg', xgb_reg), ('rf_reg', rf_reg), ('lgb_reg', lgb_reg), ('gb_reg', gb_reg)])
reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))
reg.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
###########예측

In [ ]:
er = VotingRegressor([('xgb_reg', xgb_reg), ('rf_reg', rf_reg)])
y_pred = er.fit(X_train, y_train).predict(양파_더미)
y_pred

In [ ]:
양파_더미['가격예측']=y_pred

In [ ]:
양파_더미.head()

In [ ]:
양파_더미.to_excel('양파_더미(예측).xlsx')

In [ ]:
abc = []
for alg in er.named_estimators:
    clf = er.named_estimators[alg]
    a = clf.__class__.__name__
    b = [pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])]
    abc.append({a:b})

abc